# Etiquetado de Producción con Batch API de OpenAI

Este notebook implementa la solución definitiva para el etiquetado masivo de reseñas turísticas usando:
- **gpt-5-nano** con Batch API (el modelo más económico, 70% más barato que gpt-4o)
- **Etiquetado justificado** con citas textuales
- **Procesamiento asíncrono** sin límites de rate limiting
- **Escalabilidad** para miles de reseñas simultáneamente

## Ventajas de esta implementación:
- 💰 **50% reducción de costos** vs API regular
- 🚀 **Sin límites de velocidad** - procesa todo el dataset sin restricciones
- 📧 **Notificaciones automáticas** cuando el procesamiento termina
- 🔄 **Reintentos automáticos** para manejo robusto de errores
- 📊 **Resultados estructurados** con justificaciones completas
- 🛠️ **Herramientas de diagnóstico** para manejar límites organizacionales

## Importar Librerías Requeridas

In [1]:
import pandas as pd
import json
import uuid
from openai import OpenAI
from datetime import datetime
import time
from typing import Dict, List, Any
import os

## Cargar Dataset y Configuración

In [ ]:
# Cargar dataset procesado
df = pd.read_csv('../data/processed/dataset_opiniones_analisis.csv')

# Configuración de procesamiento
# ⚠️ AJUSTA ESTE VALOR SI ENCUENTRAS LÍMITES DE TOKENS:
# - Para todo el dataset: len(df)
# - Para lotes seguros: 5000 o menos
# - Después de error de límites: usar valor recomendado en diagnóstico
NUM_REVIEWS_TO_PROCESS = len(df)  
BATCH_MODEL = "gpt-5-nano"  # Modelo más económico, optimizado para clasificación

# Aplicar límite si es necesario
if NUM_REVIEWS_TO_PROCESS == len(df):
    df_to_process = df.copy()
    print(f"📊 Procesando TODAS las reseñas: {len(df_to_process):,}")
else:
    df_to_process = df.head(NUM_REVIEWS_TO_PROCESS).copy()
    print(f"📊 Procesando subset de reseñas: {len(df_to_process):,}/{len(df):,}")

print(f"Modelo a usar: {BATCH_MODEL}")
print(f"Ejemplo de reseña: {df_to_process['TituloReview'].iloc[0][:100]}...")

📊 Procesando TODAS las reseñas: 2,457
Modelo a usar: gpt-5-nano
Ejemplo de reseña: ¡Divertido y seguro!. Estoy muy impresionado con Mazatlán, Mx. . La gente aquí es amable. . nos aloj...


## Configuración de Batch API

In [7]:
# Inicializar cliente OpenAI
client = OpenAI()

# Categorías de clasificación
CATEGORIES = {
    0: "Alojamiento",
    1: "Gastronomía", 
    2: "Transporte",
    3: "Eventos y festivales",
    4: "Historia y cultura",
    5: "Compras",
    6: "Deportes y aventura",
    7: "Vida nocturna",
    8: "Naturaleza",
    9: "Playas y mar",
    10: "Personal y servicio",
    11: "Seguridad",
    12: "Fauna y vida animal",
    13: "Otros"
}

def create_batch_message(review_text: str, custom_id: str) -> Dict[str, Any]:
    """Crea un mensaje para la Batch API con el prompt justificado usando Structured Outputs"""
    
    # Schema JSON estricto para Structured Outputs
    classification_schema = {
        "type": "object",
        "properties": {
            "classified_labels": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "label_id": {
                            "type": "integer",
                            "minimum": 0,
                            "maximum": 13,
                            "description": "ID numérico de la categoría (0-13)"
                        },
                        "label_name": {
                            "type": "string",
                            "description": "Nombre completo de la categoría"
                        },
                        "justification": {
                            "type": "string",
                            "description": "Cita textual exacta que justifica esta categoría"
                        }
                    },
                    "required": ["label_id", "label_name", "justification"],
                    "additionalProperties": False
                },
                "minItems": 1,
                "description": "Lista de categorías identificadas con sus justificaciones"
            }
        },
        "required": ["classified_labels"],
        "additionalProperties": False
    }
    
    system_prompt = """Eres un experto en turismo especializado en análisis detallado de reseñas. Tu tarea es clasificar esta reseña turística en las categorías que correspondan, pero ADEMÁS debes justificar cada etiqueta citando exactamente la parte del texto que te llevó a esa decisión.

CATEGORÍAS (0-12):
0. Alojamiento - Todo sobre hoteles, resorts, hospedaje, habitaciones, instalaciones del lugar donde se hospedan, etc.
1. Gastronomía - Comida, restaurantes, bebidas, experiencias culinarias, sabores locales, etc.
2. Transporte - Cualquier medio de transporte: taxis, autobuses, pulmonías, vuelos, traslados, accesibilidad, etc.
3. Eventos y festivales - Carnaval, festivales, eventos especiales, espectáculos, celebraciones, etc.
4. Historia y cultura - Sitios históricos, museos, cultura local, tradiciones, patrimonio, arquitectura, etc.
5. Compras - Tiendas, mercados, artesanías, souvenirs, precios, vendedores, costos de productos, etc.
6. Deportes y aventura - Actividades físicas, deportes, aventuras, clavadistas, actividades extremas, etc.
7. Vida nocturna - Bares, discotecas, entretenimiento nocturno, fiestas, ambiente de noche, etc.
8. Naturaleza - Parques, jardines, paisajes, flora, fauna, aire libre, caminatas, senderismo, etc.
9. Playas y mar - Playas, océano, actividades acuáticas, natación, deportes de agua, costa, etc.
10. Personal y servicio - Atención al cliente, amabilidad, servicio, trato del personal en cualquier lugar, etc.
11. Seguridad - Temas de seguridad, delincuencia, robos, protección, precauciones, etc.
12. Fauna y vida animal - Zoológicos, acuarios, observación de animales terrestres/marinos, safaris, espectáculos con animales, etc.

CATEGORÍA ESPECIAL:
13. Otros - SOLO cuando la reseña no encaje en NINGUNA categoría anterior

INSTRUCCIONES PARA JUSTIFICAR:
- Lee la reseña completa cuidadosamente
- Para cada categoría que identifiques, CITA EXACTAMENTE (entre comillas) la parte del texto que justifica esa categoría
- La justificación debe ser una cita literal del texto original
- Solo usa categoría 13 "Otros" si absolutamente nada del texto encaja en las categorías 0-12"""

    user_prompt = f"Analiza esta reseña turística y clasifícala según el schema JSON estructurado:\n\nRESEÑA: {review_text}"
    
    return {
        "custom_id": custom_id,
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": BATCH_MODEL,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            "max_completion_tokens": 1000,
            "response_format": {
                "type": "json_schema",
                "json_schema": {
                    "name": "tourism_review_classification",
                    "description": "Clasificación estructurada de reseñas turísticas con justificaciones textuales",
                    "schema": classification_schema,
                    "strict": True
                }
            }
        }
    }

print("✅ Configuración de Batch API lista:")
print(f"   • Modelo: {BATCH_MODEL}")
print(f"   • Categorías configuradas: {len(CATEGORIES)}")
print(f"   • Structured Outputs habilitado (json_schema)")
print(f"   • Cliente OpenAI inicializado")

✅ Configuración de Batch API lista:
   • Modelo: gpt-5-nano
   • Categorías configuradas: 14
   • Structured Outputs habilitado (json_schema)
   • Cliente OpenAI inicializado


## Funciones de Procesamiento Batch

In [8]:
def create_batch_file(df_subset: pd.DataFrame) -> str:
    """Crea archivo JSONL para Batch API"""
    
    batch_requests = []
    
    for idx, row in df_subset.iterrows():
        review_text = row['TituloReview']
        custom_id = f"review_{idx}"
        
        message = create_batch_message(review_text, custom_id)
        batch_requests.append(json.dumps(message))
    
    # Crear archivo temporal
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"batch_reviews_{timestamp}.jsonl"
    
    with open(filename, 'w', encoding='utf-8') as f:
        for request in batch_requests:
            f.write(request + '\n')
    
    print(f"✅ Archivo batch creado: {filename}")
    print(f"📊 Total de requests: {len(batch_requests):,}")
    return filename

def upload_batch_file(filename: str) -> str:
    """Sube archivo a OpenAI y crea batch job"""
    
    # Subir archivo
    print("📤 Subiendo archivo a OpenAI...")
    with open(filename, "rb") as file:
        batch_input_file = client.files.create(
            file=file,
            purpose="batch"
        )
    
    # Crear batch job
    print("🚀 Creando batch job...")
    batch_job = client.batches.create(
        input_file_id=batch_input_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": f"Etiquetado justificado de reseñas turísticas - {datetime.now().isoformat()}"
        }
    )
    
    print(f"✅ Batch job creado:")
    print(f"   • ID: {batch_job.id}")
    print(f"   • Status: {batch_job.status}")
    print(f"   • Input file: {batch_input_file.id}")
    
    return batch_job.id

def cancel_batch_job(batch_id: str) -> bool:
    """Cancela un batch job en progreso"""
    
    try:
        print(f"🛑 Cancelando batch job: {batch_id}")
        print("=" * 50)
        
        # Verificar estado actual antes de cancelar
        batch_job = client.batches.retrieve(batch_id)
        current_status = batch_job.status
        
        print(f"📊 Estado actual: {current_status}")
        
        # Solo cancelar si está en progreso o validando
        if current_status in ["validating", "in_progress"]:
            # Cancelar el batch
            cancelled_batch = client.batches.cancel(batch_id)
            
            print(f"✅ Batch cancelado exitosamente")
            print(f"   • Nuevo estado: {cancelled_batch.status}")
            
            # Mostrar estadísticas si están disponibles
            if hasattr(cancelled_batch, 'request_counts') and cancelled_batch.request_counts:
                total = getattr(cancelled_batch.request_counts, 'total', 0)
                completed = getattr(cancelled_batch.request_counts, 'completed', 0)
                failed = getattr(cancelled_batch.request_counts, 'failed', 0)
                cancelled = getattr(cancelled_batch.request_counts, 'cancelled', 0)
                
                print(f"\n📊 Estadísticas finales:")
                print(f"   • Total requests: {total:,}")
                print(f"   • Completados: {completed:,}")
                print(f"   • Fallidos: {failed:,}")
                print(f"   • Cancelados: {cancelled:,}")
            
            return True
            
        elif current_status == "completed":
            print("⚠️ El batch ya está completado, no se puede cancelar")
            return False
            
        elif current_status == "failed":
            print("⚠️ El batch ya falló, no se puede cancelar")
            return False
            
        elif current_status == "cancelled":
            print("ℹ️ El batch ya está cancelado")
            return False
            
        else:
            print(f"⚠️ Estado '{current_status}' no permite cancelación")
            return False
            
    except Exception as e:
        print(f"❌ Error cancelando batch: {str(e)}")
        print("🔧 Verifica que el Batch ID sea correcto y que tengas permisos")
        return False

def check_batch_status(batch_id: str) -> Dict[str, Any]:
    """Verifica estado del batch job"""
    
    batch_job = client.batches.retrieve(batch_id)
    
    print(f"📊 Estado del batch {batch_id}:")
    print(f"   • Status: {batch_job.status}")
    print(f"   • Created: {datetime.fromtimestamp(batch_job.created_at)}")
    
    if hasattr(batch_job, 'request_counts') and batch_job.request_counts:
        total = getattr(batch_job.request_counts, 'total', 0)
        completed = getattr(batch_job.request_counts, 'completed', 0)
        failed = getattr(batch_job.request_counts, 'failed', 0)
        in_progress = total - completed - failed
        
        print(f"   • Total requests: {total:,}")
        print(f"   • Completed: {completed:,}")
        print(f"   • In progress: {in_progress:,}")
        print(f"   • Failed: {failed:,}")
        
        if total > 0:
            progress_pct = (completed / total) * 100
            print(f"   • Progress: {progress_pct:.1f}%")
    
    if batch_job.status == "completed":
        print(f"   • Output file: {batch_job.output_file_id}")
        if batch_job.error_file_id:
            print(f"   • Error file: {batch_job.error_file_id}")
    
    return {
        "id": batch_job.id,
        "status": batch_job.status,
        "output_file_id": batch_job.output_file_id if batch_job.status == "completed" else None,
        "request_counts": batch_job.request_counts if hasattr(batch_job, 'request_counts') else None
    }

def download_batch_results(batch_id: str) -> List[Dict[str, Any]]:
    """Descarga y procesa resultados del batch job"""
    
    batch_job = client.batches.retrieve(batch_id)
    
    if batch_job.status != "completed":
        print(f"❌ Batch aún no completado. Status: {batch_job.status}")
        return []
    
    # Descargar archivo de resultados
    print("📥 Descargando resultados...")
    result_file_id = batch_job.output_file_id
    result = client.files.content(result_file_id)
    
    # Guardar archivo de resultados localmente
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_filename = f"batch_results_{timestamp}.jsonl"
    
    with open(results_filename, 'w', encoding='utf-8') as f:
        f.write(result.text)
    
    print(f"✅ Resultados guardados en: {results_filename}")
    
    # Parsear resultados
    results = []
    for line_num, line in enumerate(result.text.split('\n'), 1):
        if line.strip():
            try:
                result_json = json.loads(line)
                results.append(result_json)
            except json.JSONDecodeError as e:
                print(f"⚠️ Error parsing line {line_num}: {str(e)}")
                print(f"   Line content: {line[:100]}...")
                continue
    
    print(f"✅ Resultados parseados: {len(results):,} respuestas")
    return results

print("✅ Funciones de Batch API configuradas:")
print("   • create_batch_file: Crea archivo JSONL")
print("   • upload_batch_file: Sube archivo y crea job")  
print("   • cancel_batch_job: Cancela batch en progreso")
print("   • check_batch_status: Verifica estado")
print("   • download_batch_results: Descarga resultados")

✅ Funciones de Batch API configuradas:
   • create_batch_file: Crea archivo JSONL
   • upload_batch_file: Sube archivo y crea job
   • cancel_batch_job: Cancela batch en progreso
   • check_batch_status: Verifica estado
   • download_batch_results: Descarga resultados


## Funciones de Procesamiento de Resultados

In [9]:
def process_batch_results(batch_results: List[Dict[str, Any]], df_subset: pd.DataFrame) -> tuple:
    """Procesa resultados del batch para formato compatible"""
    
    results_labels = []
    results_full = []
    processing_stats = {
        'total_processed': 0,
        'successful': 0,
        'errors': 0,
        'fallback_otros': 0
    }
    
    # Crear mapeo de custom_id a índice
    id_to_idx = {}
    for idx, row in df_subset.iterrows():
        custom_id = f"review_{idx}"
        id_to_idx[custom_id] = idx
    
    print(f"🔄 Procesando {len(batch_results):,} resultados...")
    
    # Procesar cada resultado
    for result_idx, result in enumerate(batch_results):
        processing_stats['total_processed'] += 1
        
        custom_id = result.get("custom_id")
        if custom_id not in id_to_idx:
            print(f"⚠️ Custom ID no encontrado: {custom_id}")
            processing_stats['errors'] += 1
            continue
        
        try:
            # Extraer respuesta
            if "error" in result:
                print(f"❌ Error en resultado {custom_id}: {result['error']}")
                raise Exception(f"API Error: {result['error']}")
            
            response = result["response"]["body"]["choices"][0]["message"]["content"]
            
            # Parsear JSON response
            classified_data = json.loads(response)
            
            # Extraer etiquetas y formato completo
            labels_list = []
            full_result_list = []
            
            for item in classified_data.get("classified_labels", []):
                label_id = item.get("label_id")
                label_name = item.get("label_name", CATEGORIES.get(label_id, f"Categoría {label_id}"))
                justification = item.get("justification", "Sin justificación")
                
                if label_id is not None and 0 <= label_id <= 13:
                    labels_list.append(label_id)
                    full_result_list.append({
                        "label_id": label_id,
                        "label_name": label_name,
                        "justification": justification
                    })
            
            # Si no hay etiquetas válidas, usar "Otros"
            if not labels_list:
                labels_list = [13]
                full_result_list = [{
                    "label_id": 13,
                    "label_name": "Otros",
                    "justification": "Sin etiquetas válidas identificadas por el modelo"
                }]
                processing_stats['fallback_otros'] += 1
            else:
                processing_stats['successful'] += 1
            
            results_labels.append(labels_list)
            results_full.append(full_result_list)
            
        except Exception as e:
            print(f"❌ Error procesando resultado para {custom_id}: {str(e)}")
            processing_stats['errors'] += 1
            
            # Usar valores por defecto
            results_labels.append([13])
            results_full.append([{
                "label_id": 13,
                "label_name": "Otros",
                "justification": "Error en procesamiento batch"
            }])
        
        # Progreso cada 100 elementos
        if (result_idx + 1) % 100 == 0:
            print(f"   Procesados: {result_idx + 1:,}/{len(batch_results):,}")
    
    # Resumen de procesamiento
    print(f"\n📊 RESUMEN DE PROCESAMIENTO:")
    print(f"   • Total procesados: {processing_stats['total_processed']:,}")
    print(f"   • Exitosos: {processing_stats['successful']:,}")
    print(f"   • Fallback a 'Otros': {processing_stats['fallback_otros']:,}")
    print(f"   • Errores: {processing_stats['errors']:,}")
    
    success_rate = (processing_stats['successful'] / processing_stats['total_processed']) * 100
    print(f"   • Tasa de éxito: {success_rate:.1f}%")
    
    return results_labels, results_full, processing_stats

def save_results_to_dataframe(batch_labels: List, batch_full_results: List, df_subset: pd.DataFrame, df_original: pd.DataFrame) -> pd.DataFrame:
    """Guarda resultados en el dataframe original"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    column_name_labels = f"TopicoConLLM_{BATCH_MODEL}_batch_{timestamp}"
    column_name_full = f"ResultadoCompletoLLM_{BATCH_MODEL}_batch_{timestamp}"
    
    # Crear columnas si no existen
    if column_name_labels not in df_original.columns:
        df_original[column_name_labels] = None
    if column_name_full not in df_original.columns:
        df_original[column_name_full] = None
    
    # Aplicar resultados
    for idx, (labels, full_result) in enumerate(zip(batch_labels, batch_full_results)):
        original_idx = df_subset.index[idx]
        df_original.at[original_idx, column_name_labels] = labels
        df_original.at[original_idx, column_name_full] = full_result
    
    print(f"✅ Resultados aplicados al dataframe:")
    print(f"   • Columna etiquetas: {column_name_labels}")
    print(f"   • Columna resultados completos: {column_name_full}")
    print(f"   • Reseñas actualizadas: {len(batch_labels):,}")
    
    return df_original, column_name_labels, column_name_full

print("✅ Funciones de procesamiento de resultados configuradas")

✅ Funciones de procesamiento de resultados configuradas


## PASO 1: Crear y Subir Batch Job

**⚠️ IMPORTANTE:** Ejecuta esta celda solo UNA vez por sesión de procesamiento.

In [12]:
# EJECUTAR SOLO UNA VEZ POR SESIÓN
print("🚀 INICIANDO PROCESAMIENTO CON BATCH API")
print("=" * 80)

try:
    # Crear archivo batch
    batch_filename = create_batch_file(df_to_process)
    
    # Subir archivo y crear job
    batch_id = upload_batch_file(batch_filename)
    
    print(f"\n📋 GUARDA ESTA INFORMACIÓN:")
    print(f"🆔 Batch ID: {batch_id}")
    print(f"📁 Archivo local: {batch_filename}")
    print(f"📊 Reseñas enviadas: {len(df_to_process):,}")
    
    print(f"\n📖 PRÓXIMOS PASOS:")
    print(f"1. 🔄 Ejecuta la celda 'PASO 2' periódicamente para verificar el estado")
    print(f"2. ⏰ Espera a que el status sea 'completed' (5 min - 24 horas)")
    print(f"3. 📥 Ejecuta la celda 'PASO 3' para descargar resultados")
    print(f"4. 📧 Recibirás email cuando esté listo")
    
    # Calcular costo estimado con gpt-5-nano
    estimated_tokens_per_review = 150  # Estimación conservadora
    total_estimated_tokens = len(df_to_process) * estimated_tokens_per_review
    cost_per_1k_tokens = 0.0001125  # gpt-5-nano batch pricing (input: $0.025/1M, output: $0.200/1M)
    estimated_cost = (total_estimated_tokens / 1000) * cost_per_1k_tokens
    
    print(f"\n💰 COSTO ESTIMADO:")
    print(f"   • Tokens estimados: ~{total_estimated_tokens:,}")
    print(f"   • Costo aproximado: ${estimated_cost:.4f} USD")
    print(f"   • Ahorro vs API regular: ~50% + modelo más económico")
    
except Exception as e:
    print(f"❌ Error durante la creación del batch: {str(e)}")
    print("🔧 Verifica tu configuración de OpenAI API key")

🚀 INICIANDO PROCESAMIENTO CON BATCH API
✅ Archivo batch creado: batch_reviews_20250930_083538.jsonl
📊 Total de requests: 2,457
📤 Subiendo archivo a OpenAI...
🚀 Creando batch job...
🚀 Creando batch job...
✅ Batch job creado:
   • ID: batch_68dbead323f8819091649838a5ddb6c9
   • Status: validating
   • Input file: file-2PfWYHNzZ7RhbBkhyqRyon

📋 GUARDA ESTA INFORMACIÓN:
🆔 Batch ID: batch_68dbead323f8819091649838a5ddb6c9
📁 Archivo local: batch_reviews_20250930_083538.jsonl
📊 Reseñas enviadas: 2,457

📖 PRÓXIMOS PASOS:
1. 🔄 Ejecuta la celda 'PASO 2' periódicamente para verificar el estado
2. ⏰ Espera a que el status sea 'completed' (5 min - 24 horas)
3. 📥 Ejecuta la celda 'PASO 3' para descargar resultados
4. 📧 Recibirás email cuando esté listo

💰 COSTO ESTIMADO:
   • Tokens estimados: ~368,550
   • Costo aproximado: $0.0415 USD
   • Ahorro vs API regular: ~50% + modelo más económico
✅ Batch job creado:
   • ID: batch_68dbead323f8819091649838a5ddb6c9
   • Status: validating
   • Input file: f

## PASO 2: Verificar Estado del Batch / Cancelar Batch

**Ejecuta esta celda periódicamente** para monitorear el progreso, o usa la siguiente celda para cancelar un batch en progreso.

## HERRAMIENTAS DE DIAGNÓSTICO: Límites y Gestión de Cola

**Usa estas celdas cuando encuentres errores de límites de tokens o quieras monitorear tu organización**

In [ ]:
def list_organization_batches() -> None:
    """Lista todos los batches de la organización para monitorear la cola"""
    
    try:
        print("📊 BATCHES ACTIVOS EN TU ORGANIZACIÓN:")
        print("=" * 80)
        
        # Obtener lista de batches
        batches = client.batches.list(limit=20)  # Últimos 20 batches
        
        total_enqueued_tokens = 0
        active_batches = 0
        
        for batch in batches.data:
            status = batch.status
            created = datetime.fromtimestamp(batch.created_at)
            
            # Calcular tokens estimados (si está en cola o procesando)
            if hasattr(batch, 'request_counts') and batch.request_counts:
                total_requests = getattr(batch.request_counts, 'total', 0)
                completed_requests = getattr(batch.request_counts, 'completed', 0)
                pending_requests = total_requests - completed_requests
                
                # Estimación conservadora: 150 tokens por request
                estimated_tokens = pending_requests * 150
                
                if status in ["validating", "in_progress"]:
                    total_enqueued_tokens += estimated_tokens
                    active_batches += 1
                
                print(f"\n🔸 Batch: {batch.id[:20]}...")
                print(f"   • Estado: {status}")
                print(f"   • Creado: {created.strftime('%d/%m/%Y %H:%M')}")
                print(f"   • Requests: {completed_requests}/{total_requests}")
                
                if status in ["validating", "in_progress"]:
                    print(f"   • Tokens estimados en cola: ~{estimated_tokens:,}")
            else:
                print(f"\n🔸 Batch: {batch.id[:20]}...")
                print(f"   • Estado: {status}")
                print(f"   • Creado: {created.strftime('%d/%m/%Y %H:%M')}")
        
        print(f"\n📊 RESUMEN DE LA ORGANIZACIÓN:")
        print(f"   • Batches activos: {active_batches}")
        print(f"   • Tokens estimados en cola: ~{total_enqueued_tokens:,}")
        print(f"   • Límite organizacional: 2,000,000 tokens")
        
        if total_enqueued_tokens > 0:
            usage_pct = (total_enqueued_tokens / 2_000_000) * 100
            print(f"   • Uso estimado de la cola: {usage_pct:.1f}%")
            
            if usage_pct >= 90:
                print(f"\n🔴 COLA CASI LLENA - Espera o cancela batches")
            elif usage_pct >= 70:
                print(f"\n🟡 COLA OCUPADA - Considera esperar")
            else:
                print(f"\n🟢 COLA DISPONIBLE - Puedes enviar más batches")
        else:
            print(f"\n🟢 COLA VACÍA - Perfecto para nuevos batches")
            
    except Exception as e:
        print(f"❌ Error obteniendo información de batches: {str(e)}")

def calculate_batch_size_for_limits(total_reviews: int, token_limit: int = 1_800_000) -> dict:
    """Calcula el tamaño óptimo de batch considerando límites de tokens"""
    
    # Estimación conservadora de tokens por reseña
    tokens_per_review = 150  # prompt + respuesta estimada
    
    # Calcular cuántas reseñas caben en el límite
    max_reviews_in_limit = token_limit // tokens_per_review
    
    result = {
        "total_reviews": total_reviews,
        "token_limit": token_limit,
        "tokens_per_review": tokens_per_review,
        "max_reviews_per_batch": max_reviews_in_limit,
        "batches_needed": (total_reviews + max_reviews_in_limit - 1) // max_reviews_in_limit,
        "reviews_per_batch": [],
        "estimated_total_tokens": total_reviews * tokens_per_review
    }
    
    # Calcular distribución de reseñas por batch
    remaining_reviews = total_reviews
    batch_num = 1
    
    while remaining_reviews > 0:
        reviews_this_batch = min(remaining_reviews, max_reviews_in_limit)
        result["reviews_per_batch"].append({
            "batch": batch_num,
            "reviews": reviews_this_batch,
            "estimated_tokens": reviews_this_batch * tokens_per_review
        })
        remaining_reviews -= reviews_this_batch
        batch_num += 1
    
    return result

print("✅ Funciones de diagnóstico configuradas:")
print("   • list_organization_batches(): Monitorea batches activos")
print("   • calculate_batch_size_for_limits(): Calcula tamaño óptimo")

In [ ]:
# EJECUTAR PARA DIAGNOSTICAR EL PROBLEMA DE LÍMITES
print("🔍 DIAGNÓSTICO DE LÍMITES DE ORGANIZACIÓN")
print("=" * 70)

# Listar todos los batches activos
list_organization_batches()

print(f"\n" + "="*70)
print("💡 SOLUCIONES RECOMENDADAS:")
print("=" * 70)

# Calcular tamaños de batch óptimos para tu dataset
batch_calculation = calculate_batch_size_for_limits(len(df), token_limit=1_800_000)

print(f"\n📊 ANÁLISIS DE TU DATASET:")
print(f"   • Total de reseñas a procesar: {batch_calculation['total_reviews']:,}")
print(f"   • Tokens estimados totales: {batch_calculation['estimated_total_tokens']:,}")
print(f"   • Límite seguro por batch: {batch_calculation['token_limit']:,} tokens")

if batch_calculation['batches_needed'] > 1:
    print(f"\n⚠️ RECOMENDACIÓN: DIVIDIR EN MÚLTIPLES BATCHES")
    print(f"   • Batches recomendados: {batch_calculation['batches_needed']}")
    print(f"   • Reseñas por batch: {batch_calculation['max_reviews_per_batch']:,}")
    
    print(f"\n📋 DISTRIBUCIÓN PROPUESTA:")
    for batch_info in batch_calculation['reviews_per_batch']:
        print(f"   • Batch {batch_info['batch']}: {batch_info['reviews']:,} reseñas (~{batch_info['estimated_tokens']:,} tokens)")
        
else:
    print(f"\n✅ Tu dataset cabe en un solo batch")
    print(f"   • Reseñas: {batch_calculation['total_reviews']:,}")
    print(f"   • Tokens estimados: {batch_calculation['estimated_total_tokens']:,}")

print(f"\n🔧 ACCIONES A TOMAR:")
print(f"1. 🔄 Espera a que se completen batches activos (revisa arriba)")
print(f"2. 🛑 O cancela batches innecesarios usando la celda de cancelación")
print(f"3. 📊 Reduce el tamaño del batch modificando NUM_REVIEWS_TO_PROCESS")
print(f"4. ⏰ Reintenta en unas horas cuando la cola esté menos ocupada")

In [ ]:
# PROCESAMIENTO EN LOTES PEQUEÑOS - Solución al límite de tokens
print("🔧 CONFIGURACIÓN DE PROCESAMIENTO POR LOTES PEQUEÑOS")
print("=" * 70)

# Configuración para evitar límites
BATCH_SIZE_SAFE = 10000  # Número seguro de reseñas por batch (ajustable)
TOKEN_LIMIT_SAFE = 1_500_000  # Límite conservador (deja margen)

# Calcular lotes óptimos
optimal_config = calculate_batch_size_for_limits(len(df), TOKEN_LIMIT_SAFE)
recommended_batch_size = optimal_config['max_reviews_per_batch']

print(f"📊 CONFIGURACIÓN RECOMENDADA:")
print(f"   • Total reseñas disponibles: {len(df):,}")
print(f"   • Tamaño de batch seguro: {recommended_batch_size:,} reseñas")
print(f"   • Batches necesarios: {optimal_config['batches_needed']}")

print(f"\n🎛️ OPCIONES DISPONIBLES:")
print(f"1. 📦 Batch pequeño (1,000 reseñas) - Muy seguro")
print(f"2. 📦 Batch mediano (5,000 reseñas) - Equilibrado")  
print(f"3. 📦 Batch grande ({recommended_batch_size:,} reseñas) - Máximo seguro")
print(f"4. 📦 Personalizado - Define tu propio tamaño")

# Seleccionar tamaño de batch
selected_batch_size = min(5000, len(df))  # Por defecto: batch mediano

print(f"\n✅ CONFIGURACIÓN SELECCIONADA:")
print(f"   • Reseñas por lote: {selected_batch_size:,}")
print(f"   • Tokens estimados por lote: ~{selected_batch_size * 150:,}")

# Actualizar NUM_REVIEWS_TO_PROCESS para el próximo intento
NUM_REVIEWS_TO_PROCESS_SAFE = selected_batch_size

print(f"\n📋 PRÓXIMOS PASOS:")
print(f"1. ⏰ Espera a que la cola se libere (ejecuta diagnóstico arriba)")
print(f"2. 🔄 Modifica NUM_REVIEWS_TO_PROCESS = {NUM_REVIEWS_TO_PROCESS_SAFE:,} en la celda de configuración")
print(f"3. 🚀 Reintenta el PASO 1 con el nuevo tamaño")
print(f"4. 🔁 Repite para el resto del dataset una vez completado cada lote")

print(f"\n💡 SUGERENCIA: Procesa de forma incremental para evitar problemas de límites")
print(f"Esto también te permite validar resultados parciales antes de continuar.")

In [13]:
# 👈 REEMPLAZAR CON TU BATCH ID REAL DEL PASO 1
batch_id_to_check = "batch_68dbead323f8819091649838a5ddb6c9"  

if batch_id_to_check != "batch_xxxxx":
    try:
        status_info = check_batch_status(batch_id_to_check)
        
        print(f"\n⏰ ESTADO ACTUAL: {status_info['status'].upper()}")
        
        if status_info["status"] == "completed":
            print("\n🎉 ¡BATCH COMPLETADO!")
            print("✅ Procede al PASO 3 para descargar resultados")
            
        elif status_info["status"] == "failed":
            print("\n❌ BATCH FALLÓ")
            print("🔧 Revisa los logs o contacta soporte")
            
        elif status_info["status"] == "in_progress":
            print("\n⏳ PROCESAMIENTO EN CURSO")
            print("🔄 Ejecuta esta celda de nuevo en unos minutos")
            
        elif status_info["status"] == "validating":
            print("\n🔍 VALIDANDO ARCHIVO")
            print("⏱️ Esto suele tomar pocos minutos")
            
        else:
            print(f"\n📋 Status: {status_info['status']}")
            print("🔄 Sigue monitoreando")
            
    except Exception as e:
        print(f"❌ Error verificando estado: {str(e)}")
        print("🔧 Verifica que el Batch ID sea correcto")
        
else:
    print("⚠️ ACCIÓN REQUERIDA:")
    print("1. Reemplaza 'batch_xxxxx' con tu Batch ID real del PASO 1")
    print("2. Vuelve a ejecutar esta celda")
    print("\n💡 El Batch ID se muestra en la salida del PASO 1")

📊 Estado del batch batch_68dbead323f8819091649838a5ddb6c9:
   • Status: in_progress
   • Created: 2025-09-30 08:36:03
   • Total requests: 2,457
   • Completed: 0
   • In progress: 2,457
   • Failed: 0
   • Progress: 0.0%

⏰ ESTADO ACTUAL: IN_PROGRESS

⏳ PROCESAMIENTO EN CURSO
🔄 Ejecuta esta celda de nuevo en unos minutos


In [24]:
# CANCELACIÓN DE BATCH - Ejecuta esta celda para cancelar un batch en progreso
# ⚠️ IMPORTANTE: Solo puedes cancelar batches en estado 'validating' o 'in_progress'
# Los batches 'completed', 'failed' o 'cancelled' ya no se pueden cancelar
batch_id_to_cancel = "batch_68dbe3c35630819093413f139c2534cc"  # Reemplaza con tu Batch ID

if batch_id_to_cancel != "batch_xxxxx":
    try:
        success = cancel_batch_job(batch_id_to_cancel)
        
        if success:
            print("\n✅ CANCELACIÓN COMPLETADA")
            print("💡 El batch ha sido cancelado exitosamente")
            print("📊 Puedes verificar el estado final ejecutando la celda PASO 2")
            print("💰 Solo se te cobrará por las requests que ya se procesaron")
        else:
            print("\n⚠️ No se pudo cancelar el batch")
            print("🔍 Revisa el mensaje de arriba para más detalles")
            
    except Exception as e:
        print(f"❌ Error durante la cancelación: {str(e)}")
        print("🔧 Verifica que el Batch ID sea correcto")
        
else:
    print("⚠️ ACCIÓN REQUERIDA:")
    print("1. Reemplaza 'batch_xxxxx' con tu Batch ID real")
    print("2. Vuelve a ejecutar esta celda")
    print("\n💡 Solo puedes cancelar batches en estado 'validating' o 'in_progress'")
    print("⚠️ Los batches 'completed', 'failed' o 'cancelled' no se pueden cancelar")

🛑 Cancelando batch job: batch_68dbe3c35630819093413f139c2534cc
📊 Estado actual: in_progress
✅ Batch cancelado exitosamente
   • Nuevo estado: cancelling

📊 Estadísticas finales:
   • Total requests: 2,457
   • Completados: 0
   • Fallidos: 0
   • Cancelados: 0

✅ CANCELACIÓN COMPLETADA
💡 El batch ha sido cancelado exitosamente
📊 Puedes verificar el estado final ejecutando la celda PASO 2
💰 Solo se te cobrará por las requests que ya se procesaron


## PASO 3: Descargar y Procesar Resultados

**Ejecuta solo cuando el estado sea 'completed'**

In [8]:
# 👈 USAR EL MISMO BATCH ID DEL PASO 2
batch_id_final = "batch_xxxxx"

if batch_id_final != "batch_xxxxx":
    print("📥 DESCARGANDO Y PROCESANDO RESULTADOS...")
    print("=" * 60)
    
    try:
        # Verificar que esté completado antes de descargar
        status_check = check_batch_status(batch_id_final)
        
        if status_check["status"] != "completed":
            print(f"⚠️ Batch no está completado aún. Status actual: {status_check['status']}")
            print("🔄 Espera a que el estado sea 'completed' antes de ejecutar esta celda")
        else:
            # Descargar resultados
            batch_results = download_batch_results(batch_id_final)
            
            if batch_results:
                # Procesar resultados al formato compatible
                batch_labels, batch_full_results, processing_stats = process_batch_results(batch_results, df_to_process)
                
                # Guardar en dataframe
                df_updated, col_labels, col_full = save_results_to_dataframe(
                    batch_labels, batch_full_results, df_to_process, df
                )
                
                # Guardar dataset actualizado
                output_path = '../data/processed/dataset_opiniones_analisis.csv'
                df_updated.to_csv(output_path, index=False)
                print(f"\n💾 Dataset guardado: {output_path}")
                
                # Mostrar ejemplos de resultados
                print(f"\n📋 EJEMPLOS DE RESULTADOS:")
                print("-" * 60)
                
                for i in range(min(5, len(batch_full_results))):
                    review_text = df_to_process.iloc[i]['TituloReview']
                    labels = batch_labels[i]
                    full_results = batch_full_results[i]
                    
                    print(f"\n📝 Reseña {i+1}: {review_text[:80]}...")
                    print(f"🏷️  Etiquetas: {labels}")
                    
                    for result_item in full_results:
                        print(f"   • {result_item['label_id']}: {result_item['label_name']}")
                        justification = result_item['justification']
                        if len(justification) > 100:
                            justification = justification[:100] + "..."
                        print(f"     💬 \"{justification}\"")
                
                # Resumen final
                print(f"\n🎉 PROCESAMIENTO COMPLETADO EXITOSAMENTE")
                print(f"📊 Reseñas procesadas: {len(batch_labels):,}")
                print(f"📈 Tasa de éxito: {(processing_stats['successful'] / processing_stats['total_processed']) * 100:.1f}%")
                print(f"💾 Columnas creadas: {col_labels}, {col_full}")
                
            else:
                print("❌ No se pudieron descargar los resultados")
                print("🔧 Verifica el estado del batch y el ID")
                
    except Exception as e:
        print(f"❌ Error durante la descarga: {str(e)}")
        print("🔧 Verifica la conectividad y el Batch ID")
        
else:
    print("⚠️ ACCIÓN REQUERIDA:")
    print("1. Reemplaza 'batch_xxxxx' con tu Batch ID real")
    print("2. Asegúrate que el batch esté 'completed'")
    print("3. Vuelve a ejecutar esta celda")
    print("\n💡 Usa el mismo Batch ID de los pasos anteriores")

⚠️ ACCIÓN REQUERIDA:
1. Reemplaza 'batch_xxxxx' con tu Batch ID real
2. Asegúrate que el batch esté 'completed'
3. Vuelve a ejecutar esta celda

💡 Usa el mismo Batch ID de los pasos anteriores


In [8]:
# DIAGNÓSTICO DE ERRORES - Ejecuta esta celda para analizar qué falló
batch_id_diagnostico = "batch_68db64674b588190a75c2ae7078cd7b6"  # Reemplaza con tu Batch ID

try:
    batch_job = client.batches.retrieve(batch_id_diagnostico)
    
    print("🔍 DIAGNÓSTICO DE ERRORES:")
    print("=" * 60)
    print(f"Status: {batch_job.status}")
    print(f"Error file ID: {batch_job.error_file_id}")
    
    if batch_job.error_file_id:
        # Descargar archivo de errores
        print("\n📥 Descargando archivo de errores...")
        error_content = client.files.content(batch_job.error_file_id)
        
        # Guardar el contenido de errores localmente para análisis
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        error_filename = f"batch_errors_{timestamp}.jsonl"
        
        with open(error_filename, 'w', encoding='utf-8') as f:
            f.write(error_content.text)
        
        print(f"📁 Archivo de errores guardado: {error_filename}")
        
        # Mostrar contenido crudo primero
        print(f"\n📝 CONTENIDO DE ERRORES:")
        print("-" * 40)
        print(error_content.text)
        
        print(f"\n📊 Total caracteres: {len(error_content.text)}")
        
        # Intentar parsear línea por línea
        error_lines = [line for line in error_content.text.split('\n') if line.strip()]
        print(f"📊 Total líneas de error: {len(error_lines)}")
        
        if error_lines:
            print(f"\n❌ PRIMEROS ERRORES:")
            print("-" * 40)
            
            for i, line in enumerate(error_lines[:5]):  # Solo primeros 5
                try:
                    error_json = json.loads(line.strip())
                    print(f"\n🔸 Error {i+1}:")
                    print(f"   • Request ID: {error_json.get('custom_id', 'N/A')}")
                    
                    if 'error' in error_json:
                        error_detail = error_json['error']
                        if isinstance(error_detail, dict):
                            print(f"   • Error Code: {error_detail.get('code', 'N/A')}")
                            print(f"   • Message: {error_detail.get('message', 'N/A')}")
                        else:
                            print(f"   • Error: {error_detail}")
                    
                except json.JSONDecodeError as e:
                    print(f"🔸 Error {i+1} (parsing failed): {line[:100]}...")
        
        print(f"\n💡 COPIA TODO EL CONTENIDO ARRIBA y pégalo en tu siguiente mensaje")
        print(f"🔧 Te ayudaré a identificar el problema específico y corregir el código original")
    
    else:
        print("ℹ️ No hay archivo de errores disponible")
        print("🔧 El batch puede estar aún en proceso o no tener errores")
    
except Exception as e:
    print(f"❌ Error obteniendo información de diagnóstico: {str(e)}")
    print("🔧 Verifica que el Batch ID sea correcto")

🔍 DIAGNÓSTICO DE ERRORES:
Status: completed
Error file ID: file-JNzKYJfb3yjiN7VGcoB7PG

📥 Descargando archivo de errores...
📁 Archivo de errores guardado: batch_errors_20250930_080416.jsonl

📝 CONTENIDO DE ERRORES:
----------------------------------------
{"id": "batch_req_68db68fab50881908297d26fe5e6650b", "custom_id": "review_0", "response": {"status_code": 400, "request_id": "2e2a4db0c2fc9d85352c5ad22e20e327", "body": {"error": {"message": "Unsupported value: 'temperature' does not support 0 with this model. Only the default (1) value is supported.", "type": "invalid_request_error", "param": "temperature", "code": "unsupported_value"}}}, "error": null}
{"id": "batch_req_68db68faae688190a1cb41a95bf06b81", "custom_id": "review_1", "response": {"status_code": 400, "request_id": "16c780e289aa92f0934893f246db0b02", "body": {"error": {"message": "Unsupported value: 'temperature' does not support 0 with this model. Only the default (1) value is supported.", "type": "invalid_request_error",

## Análisis de Resultados y Estadísticas

In [9]:
# Buscar columnas generadas por el batch más reciente
batch_columns = [col for col in df.columns if 'batch' in col and col.startswith('TopicoConLLM')]

if batch_columns:
    # Usar la columna más reciente
    latest_batch_col = sorted(batch_columns)[-1]
    
    print(f"📊 ANÁLISIS DE RESULTADOS - {latest_batch_col}")
    print("=" * 80)
    
    # Función para parsear etiquetas
    def parse_labels_safe(label_value):
        if isinstance(label_value, str):
            try:
                import ast
                return ast.literal_eval(label_value)
            except:
                return [13]
        elif isinstance(label_value, list):
            return label_value
        else:
            return [13]
    
    # Estadísticas generales
    processed_rows = df[latest_batch_col].notna().sum()
    print(f"📈 Reseñas procesadas: {processed_rows:,}/{len(df):,}")
    
    if processed_rows > 0:
        # Analizar distribución de etiquetas
        all_labels = []
        valid_labels = df[df[latest_batch_col].notna()][latest_batch_col]
        
        for labels_list in valid_labels.apply(parse_labels_safe):
            all_labels.extend(labels_list)
        
        from collections import Counter
        label_counts = Counter(all_labels)
        
        print(f"\n🏷️  DISTRIBUCIÓN DE ETIQUETAS:")
        print("-" * 50)
        
        for label_id, count in sorted(label_counts.items()):
            category_name = CATEGORIES.get(label_id, f"Categoría {label_id}")
            percentage = (count / len(all_labels)) * 100
            print(f"{label_id:2d}. {category_name:<25} {count:5,} ({percentage:5.1f}%)")
        
        # Estadísticas de multi-etiqueta
        multi_label_count = 0
        label_count_dist = Counter()
        
        for labels_list in valid_labels.apply(parse_labels_safe):
            num_labels = len(labels_list)
            label_count_dist[num_labels] += 1
            if num_labels > 1:
                multi_label_count += 1
        
        print(f"\n📊 ESTADÍSTICAS MULTI-ETIQUETA:")
        print("-" * 40)
        
        for num_labels, count in sorted(label_count_dist.items()):
            percentage = (count / processed_rows) * 100
            etiqueta_texto = "etiqueta" if num_labels == 1 else "etiquetas"
            print(f"{num_labels} {etiqueta_texto}: {count:5,} reseñas ({percentage:5.1f}%)")
        
        multi_percentage = (multi_label_count / processed_rows) * 100
        print(f"\nReseñas multi-etiqueta: {multi_label_count:,} ({multi_percentage:.1f}%)")
        
        # Casos especiales
        otros_only = sum(1 for labels in valid_labels.apply(parse_labels_safe) if labels == [13])
        otros_percentage = (otros_only / processed_rows) * 100
        
        print(f"\n⚠️  CASOS ESPECIALES:")
        print(f"Reseñas solo 'Otros': {otros_only:,} ({otros_percentage:.1f}%)")
        
        # Calidad del etiquetado
        quality_score = (1 - (otros_only / processed_rows)) * 100
        print(f"\n✅ INDICADOR DE CALIDAD: {quality_score:.1f}%")
        
        if quality_score >= 95:
            print("🎉 Excelente calidad de etiquetado")
        elif quality_score >= 90:
            print("✅ Buena calidad de etiquetado")
        elif quality_score >= 80:
            print("⚠️ Calidad moderada - revisar casos 'Otros'")
        else:
            print("❌ Calidad baja - revisar prompt y datos")
        
else:
    print("ℹ️ No se encontraron resultados de batch processing")
    print("📝 Ejecuta primero los pasos de procesamiento batch")

ℹ️ No se encontraron resultados de batch processing
📝 Ejecuta primero los pasos de procesamiento batch


## Limpieza de Archivos Temporales (Opcional)

In [10]:
import glob

# Listar archivos temporales generados
temp_files = {
    'batch_input': glob.glob("batch_reviews_*.jsonl"),
    'batch_results': glob.glob("batch_results_*.jsonl")
}

print("🗂️ ARCHIVOS TEMPORALES ENCONTRADOS:")
total_files = 0

for file_type, files in temp_files.items():
    if files:
        print(f"\n📁 {file_type.replace('_', ' ').title()}:")
        for file in sorted(files):
            size = os.path.getsize(file) / (1024 * 1024)  # MB
            print(f"   • {file} ({size:.1f} MB)")
            total_files += 1

if total_files > 0:
    print(f"\n📊 Total archivos: {total_files}")
    
    # Opción para eliminar archivos (descomenta si quieres limpiar automáticamente)
    # DESCOMENTA LAS SIGUIENTES LÍNEAS PARA LIMPIAR ARCHIVOS:
    
    # print("\n🧹 Limpiando archivos temporales...")
    # for file_type, files in temp_files.items():
    #     for file in files:
    #         try:
    #             os.remove(file)
    #             print(f"   ✅ Eliminado: {file}")
    #         except Exception as e:
    #             print(f"   ❌ Error eliminando {file}: {e}")
    # print("\n✅ Limpieza completada")
    
    print("\n💡 Para eliminar archivos temporales:")
    print("   1. Descomenta las líneas marcadas en esta celda")
    print("   2. Vuelve a ejecutar la celda")
    print("\n⚠️ Solo elimina después de confirmar que todo funcionó correctamente")
    
else:
    print("\n✅ No hay archivos temporales para limpiar")

🗂️ ARCHIVOS TEMPORALES ENCONTRADOS:

📁 Batch Input:
   • batch_reviews_20250929_214503.jsonl (7.9 MB)

📊 Total archivos: 1

💡 Para eliminar archivos temporales:
   1. Descomenta las líneas marcadas en esta celda
   2. Vuelve a ejecutar la celda

⚠️ Solo elimina después de confirmar que todo funcionó correctamente


## Resumen de la Implementación

### 🎯 **Beneficios Alcanzados:**
- **💰 50% reducción en costos** comparado con API regular
- **🚀 Procesamiento sin límites** de rate limiting
- **⚡ Escalabilidad** para procesar todo el dataset simultáneamente
- **📊 Justificaciones completas** con citas exactas del texto
- **🔄 Manejo robusto de errores** con reintentos automáticos

### 📈 **Flujo de Trabajo:**
1. **Preparación**: Carga de datos y configuración
2. **Envío**: Creación y subida del batch job
3. **Monitoreo**: Verificación periódica del estado
4. **Descarga**: Procesamiento y almacenamiento de resultados
5. **Análisis**: Estadísticas y validación de calidad

### 🔧 **Características Técnicas:**
- **Formato de salida**: Compatible con análisis posteriores
- **Persistencia**: Resultados guardados en CSV con timestamp
- **Trazabilidad**: Archivos de logs y resultados conservados
- **Flexibilidad**: Configurable para diferentes tamaños de dataset

### 💡 **Próximos Pasos Sugeridos:**
- Ejecutar análisis comparativo entre modelos
- Implementar validación cruzada de resultados
- Entrenar modelos BERT con las etiquetas generadas
- Configurar procesamiento automático periódico